# OOP

Student: Pollux Gronier


B00822392


Fevrier 2026

## Factory Pattern

In [26]:
from abc import ABC, abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def validate(self, details: dict) -> bool:
        pass
    @abstractmethod
    def process(self, amount: float, details: dict) -> dict:
        pass

class CreditCardProcessor(PaymentProcessor):
    def validate(self, details: dict) -> bool:
        num=details.get("card_number", "")
        cvv=details.get("cvv", "")
        return len(num)==16 and len(cvv)==3
    def process(self, amount: float, details: dict) -> dict:
        if not self.validate(details): return {"success": False, "error":"Invalid Card"}
        fee = round(amount*0.029, 2)
        return {"success":True, "method":"credit_card", "amount":amount+fee, "fee":fee}

class BankTransferProcessor(PaymentProcessor):
    def validate(self, details: dict) -> bool:
        return len(details.get("iban", "")) >= 15
    def process(self, amount: float, details: dict) -> dict:
        if not self.validate(details): return {"success": False, "error":"Invalid IBAN"}
        fee=1.50
        return {"success":True, "method":"bank_transfer", "amount":amount+fee, "fee":fee}

class PayPalProcessor(PaymentProcessor):
    def validate(self, details: dict) -> bool:
        return "@" in details.get("email", "")
    def process(self, amount: float, details: dict) -> dict:
        if not self.validate(details): return {"success": False, "error": "Invalid Email"}
        fee=(amount * 0.034) + 0.30
        return {"success":True, "method":"paypal", "amount":amount+fee, "fee":fee}

class PaymentFactory:
    _processors={
        "credit_card":CreditCardProcessor,
        "bank_transfer":BankTransferProcessor,
        "paypal":PayPalProcessor
    }
    def get_processor(self, payment_type:str):
        processor_class=self._processors.get(payment_type)
        if not processor_class: raise ValueError(f"Unknown type: {payment_type}")
        return processor_class()

processor_factory = PaymentFactory()

#Test differents methods
methods = [
    ("credit_card", {"card_number": "1111222233334444", "cvv": "999"}),
    ("paypal", {"email": "user@provider.com"}),
    ("bank_transfer", {"iban": "FR7612345678901234567890123"})
]

for p_type, details in methods:
    proc = processor_factory.get_processor(p_type)
    receipt = proc.process(100.0, details)
    print(f"Method:{p_type.upper()} | Total Charged:{receipt['amount']}€ (Fee:{receipt['fee']}€)")

Method:CREDIT_CARD | Total Charged:102.9€ (Fee:2.9€)
Method:PAYPAL | Total Charged:103.7€ (Fee:3.7€)
Method:BANK_TRANSFER | Total Charged:101.5€ (Fee:1.5€)


## Builder Pattern

In [27]:
from dataclasses import dataclass

@dataclass
class Employee:
    first_name:str=None
    last_name:str=None
    email:str=None
    department:str=None
    position:str=None
    salary:float=0.0
    has_laptop:bool=False
    has_parking:bool=False
    has_vpn_access:bool=False
    has_admin_rights:bool=False

    def __str__(self):
        return f"Employee:{self.first_name}{self.last_name}({self.position})"

class EmployeeBuilder:
    def __init__(self):
        self.employee=Employee()
    def with_name(self, first, last):
        self.employee.first_name=first
        self.employee.last_name=last
        return self
    def with_email(self, email):
        self.employee.email=email
        return self
    def with_job(self, dept, pos, sal):
        self.employee.department=dept
        self.employee.position=pos
        self.employee.salary=sal
        return self
    def with_equipment(self, laptop=False, parking=False):
        self.employee.has_laptop=laptop
        self.employee.has_parking=parking
        return self
    def with_access(self, vpn=False, admin=False):
        self.employee.has_vpn_access=vpn
        self.employee.has_admin_rights=admin
        return self
    def build(self):
        if not self.employee.first_name or not self.employee.last_name:
            raise ValueError("Name is required")
        if not self.employee.email:
            raise ValueError("Email is required")
        return self.employee

class DeveloperBuilder(EmployeeBuilder):
    def __init__(self, first, last, email):
        super().__init__()
        (self.with_name(first, last)
             .with_email(email)
             .with_job("Engineering", "Senior Developer", 75000)
             .with_equipment(laptop=True)
             .with_access(vpn=True, admin=True))

#Tests
worker=(EmployeeBuilder()
          .with_name("John", "Doe")
          .with_email("john.doe@company.com")
          .with_job("Engineering", "Senior Dev", 75000)
          .with_access(vpn=True, admin=True)
          .build())

dev_preset=DeveloperBuilder("Jane", "Smith", "jane.smith@company.com").build()

print(worker)
print(dev_preset)

Employee:JohnDoe(Senior Dev)
Employee:JaneSmith(Senior Developer)


## Singleton Pattern

In [28]:
import json

class ConfigManager:
    _instance=None
    _config_data={
        "app": {"name": "PaymentPlatform", "debug": True},
        "database": {"host": "localhost", "port": 5432},
        "email": {"smtp_host": "smtp.company.com", "sender": "no-reply@company.com"},
        "payment": {"api_key": "sk_test_123456", "environment": "sandbox"}
    }
    def __new__(cls):
        if cls._instance is None:
            cls._instance=super(ConfigManager, cls).__new__(cls)
        return cls._instance
    @classmethod
    def get_instance(cls):
        return cls()
    def get(self, path: str):
        keys=path.split('.')
        val=self._config_data
        for k in keys:
            if isinstance(val, dict):
                val=val.get(k)
            else:
                return None
        return val

#tests
# Create two 'different' manager references
config_prime = ConfigManager.get_instance()
config_secondary = ConfigManager.get_instance()

print(f"Are they the same object? {config_prime is config_secondary}")

# Accessing specific data using the dot notation
db_target = config_prime.get("database.host")
is_debug = config_prime.get("app.debug")

print(f"Targeting Database: {db_target}")
print(f"System in Debug Mode: {is_debug}")

Are they the same object? True
Targeting Database: localhost
System in Debug Mode: True
